# <font size=6 color='green'><center>**IT 496 - Introduction to Data Mining**</center></font>
### **<center>Course Project - 03<br/><br/>ICC Cricket World Cup 2023 ML Challange<center>**

 &emsp; **Group Name : <font color='#0099ff'>DataForce</font>** <br/>
 &emsp; **Group ID : <font color='#0099ff'>T06</font>**

In [1]:
# importing required libraries
import numpy as np
import pandas as pd

### <center>Task 1</center>

**Problems**
1. Predicting the batsman who will score most runs/ hit most sixes/ make most centuries in
the tournament.
2. Predicting the batsman who will score most runs/ hit most sixes/ make most centuries in
the tournament.

---

### <center>Task 2</center>
You are required to predict the two finalist teams in the ICC Cricket World Cup 2023 and the
players (11 players for each finalist team) who are likely to be part of these teams. This task
involves team composition prediction.

In [2]:
from teamCompositionPredictor import getTeamCompsition
from tensorflow import keras
from Helpers import getTeamsMapping,getVenuesMapping, getPlayerRole, getTeamStats

In [3]:
def getMatchWinnerPredictor():
  # loading the pre-trained model
  model = keras.models.load_model('MatchWinnerPredictor.keras')
  return model

In [4]:
additional_cols1 = ['team1 avg score','team1 avg wickets','team1 avg catches','team1 avg fours',
                   'team1 avg sixes','team1 odi rank']
additional_cols2 = ['team2 avg score','team2 avg wickets','team2 avg catches','team2 avg fours',
                   'team2 avg sixes','team2 odi rank']

Predicting for the remaining matches of ICC

In [5]:
matches = pd.DataFrame(columns=['season', 'team1', 'team2', 'Venue', 'team1 avg score',
       'team1 avg wickets', 'team1 avg catches', 'team1 avg fours',
       'team1 avg sixes', 'team1 odi rank', 'team2 avg score',
       'team2 avg wickets', 'team2 avg catches', 'team2 avg fours',
       'team2 avg sixes', 'team2 odi rank'])

In [6]:
matches.loc[0,['season','team1','team2','Venue']] = ['2023','India','New Zealand','Wankhede Stadium, Mumbai']
matches.loc[1,['season','team1','team2','Venue']] = ['2023','South Africa','Australia','Eden Gardens, Kolkata']
matches.loc[0,additional_cols1] = getTeamStats(matches.iloc[0]['team1'])
matches.loc[0,additional_cols2] = getTeamStats(matches.iloc[0]['team2'])
matches.loc[1,additional_cols1] = getTeamStats(matches.iloc[1]['team1'])
matches.loc[1,additional_cols2] = getTeamStats(matches.iloc[1]['team2'])
# teams in finals is not yet know
matches.loc[2,['season','Venue']] = ['2023','Narendra Modi Stadium, Ahmedabad']
matches.head()

,season,team1,team2,Venue,team1 avg score,team1 avg wickets,team1 avg catches,team1 avg fours,team1 avg sixes,team1 odi rank,team2 avg score,team2 avg wickets,team2 avg catches,team2 avg fours,team2 avg sixes,team2 odi rank
0,2023,India,New Zealand,"Wankhede Stadium, Mumbai",25.180872,85.37697,38.8,244.555705,55.894855,1.0,18.430865,72.762424,27.533333,121.666667,25.133333,5.0
1,2023,South Africa,Australia,"Eden Gardens, Kolkata",22.147129,64.220606,20.0,121.0,27.2,3.0,23.453834,59.333333,34.6,191.333333,47.266667,2.0
2,2023,NaN,NaN,"Narendra Modi Stadium, Ahmedabad",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
venuesMapping = getVenuesMapping()
teamsMapping = getTeamsMapping()
matches['Venue'] = matches['Venue'].apply(lambda x:venuesMapping[x])
matches.loc[0,'team1'] = teamsMapping[matches.iloc[0]['team1']]
matches.loc[0,'team2'] = teamsMapping[matches.loc[0]['team2']]
matches.loc[1,'team1'] = teamsMapping[matches.loc[1]['team1']]
matches.loc[1,'team2'] = teamsMapping[matches.loc[1]['team2']]

In [8]:
nn_model = getMatchWinnerPredictor()

In [9]:
# importing the RobustScaler 
# to normalize using scaler fitted on train data
import pickle
with open('RobustScaler.pkl','rb') as f:
    scaler = pickle.load(f)

Semi-Final 1 Prediction

In [10]:
input1 = np.asarray(matches.iloc[0])
x1 = scaler.transform(np.expand_dims(input1,axis=0))
nn_model.predict(x1)

1/1 [==============================] - 0s 108ms/step


array([[0.6995439, 0.3004561]], dtype=float32)

The Model predicted that Team 1(India) wins with probability 0.699 and Team 2(New Zealand) wins with probability 0.3.
Thus India is the winner of semi-final 1!!

Semi-Final 2 Prediction

In [11]:
input2 = np.asarray(matches.iloc[1])
x2 = scaler.transform(np.expand_dims(input2,axis=0))
nn_model.predict(x2)

1/1 [==============================] - 0s 27ms/step


array([[0.76192933, 0.23807067]], dtype=float32)

The Model predicted that Team 1(South Africa) wins with probability 0.762 and Team 2(Australia) wins with probability 0.238.
Thus South Africa is the winner of semi-final 2!!

### Team Composition Prediction

In [12]:
team1List, team2List = getTeamCompsition("India","South Africa","Narendra Modi Stadium, Ahmedabad")
print("Playing 11 of India : ")
print(team1List)
print("\nPlaying 11 of South Africa")
print(team2List)

Playing 11 of India : 
['Shubman Gill' 'Virat Kohli' 'Rohit Sharma' 'KL Rahul' 'Shreyas Iyer'
 'Ishan Kishan' 'Mohammed Shami' 'Kuldeep Yadav' 'Jasprit Bumrah'
 'Suryakumar Yadav' 'Hardik Pandya']

Playing 11 of South Africa
['Kagiso Rabada' 'Temba Bavuma (Captain)' 'Quinton de Kock' 'David Miller'
 'Aiden Markram' 'Rassie van der Dussen' 'Reeza Hendricks' 'Lungi Ngidi'
 'Tabraiz Shamsi' 'Heinrich Klaasen' 'Marco Jansen']


---

### <center>Task 3</center>
You should predict the winner of the ICC Cricket World Cup 2023.

In [13]:
matches.loc[2,['team1','team2']]=['India','South Africa']
matches.loc[2,additional_cols1] = getTeamStats(matches.iloc[2]['team1'])
matches.loc[2,additional_cols2] = getTeamStats(matches.iloc[2]['team2'])
matches.loc[2,'team1'] = teamsMapping[matches.iloc[2]['team1']]
matches.loc[2,'team2'] = teamsMapping[matches.loc[2]['team2']]
matches.head()

,season,team1,team2,Venue,team1 avg score,team1 avg wickets,team1 avg catches,team1 avg fours,team1 avg sixes,team1 odi rank,team2 avg score,team2 avg wickets,team2 avg catches,team2 avg fours,team2 avg sixes,team2 odi rank
0,2023,5,7,9,25.180872,85.37697,38.8,244.555705,55.894855,1.0,18.430865,72.762424,27.533333,121.666667,25.133333,5.0
1,2023,9,2,10,22.147129,64.220606,20.0,121.0,27.2,3.0,23.453834,59.333333,34.6,191.333333,47.266667,2.0
2,2023,5,9,1,25.180872,85.37697,38.8,244.555705,55.894855,1.0,22.147129,64.220606,20.0,121.0,27.2,3.0


In [14]:
finalInput = np.asarray(matches.iloc[2])
x = scaler.transform(np.expand_dims(finalInput,axis=0))
nn_model.predict(x)

1/1 [==============================] - 0s 23ms/step


array([[0.82144415, 0.1785559 ]], dtype=float32)

The model predicted 0.82 for Team 1(India) and 0.18 for Team 2(South Africa)

The Model predicted the final winner as India with probability 82%

---